In [1]:
import imageio
import numpy as np
from tqdm import tqdm
import tensorflow as tf
import colorizerutils as utils
from IPython.display import Video
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import initializers

In [2]:
# The amount of GPU memory to allocate to Tensorflow
vram = 14 # GB

gpus = tf.config.list_physical_devices('GPU')

tf.config.set_logical_device_configuration(
    gpus[0],
    [tf.config.LogicalDeviceConfiguration(memory_limit=vram*1024)])

tf.__version__

'2.8.0'

In [4]:
# Creates a dataset using images from ../images/train_images/
train_dataset = tf.data.Dataset.from_generator(
    lambda: utils.image_loader('../shows/trainepisodes/'),
    output_types=(tf.float32,tf.float32)
)

# Directory for tensorboard logs
log_directory = 'logs/'

# Set up tensorboard logging for the generator and discriminator loss
summary_writer = tf.summary.create_file_writer(log_directory + 'dusty/')
gen_loss_tracker = metrics.Mean('Generator_loss',dtype=tf.float32)
gen_adv_tracker = metrics.Mean('Generator_adversarial_loss',dtype=tf.float32)
gen_mse_tracker = metrics.Mean('Generator_mse_loss',dtype=tf.float32)
disc_loss_tracker = metrics.Mean('Discriminator_loss',dtype=tf.float32)

In [3]:
# Downsampling block for use in generator and discriminator
def downsampling(filters,stride,prev_layer):

    init = initializers.RandomNormal()

    block = layers.Conv2D(filters,strides=stride,kernel_size=4,padding='same',kernel_initializer=init,use_bias=False)(prev_layer)
    block = layers.BatchNormalization()(block)
    block = layers.LeakyReLU(0.2)(block)

    return block

# Upsampling block for use in generator only
# Skip layer should be the same shape as the output to the transpose convolutional layer
# Or twice the size of the prev_layer input
def upsampling(filters,stride,prev_layer,skip_layer):

    init = initializers.RandomNormal()

    block = layers.Conv2DTranspose(
        filters,strides=stride,kernel_size=4,padding='same',kernel_initializer=init,use_bias=False)(prev_layer)
    block = layers.BatchNormalization()(block)
    block = layers.Concatenate()([block,skip_layer])
    block = layers.LeakyReLU(0.2)(block)
    block = layers.Dropout(0.3)(block)

    return block

In [5]:
def make_generator():

    init = initializers.RandomNormal()

    # Model input is a (64x64) grayscale image
    model_input = layers.Input(shape=(64,64,1))

    # Downsampling stack
    down0 = downsampling(filters=32,stride=1,prev_layer=model_input) # (64x64) -> (64x64)
    down1 = downsampling(64,2,down0) # (64x64) -> (32x32)
    down2 = downsampling(128,2,down1) # (32x32) -> (16x16)
    down3 = downsampling(256,2,down2) # (16x16) -> (8x8)
    down4 = downsampling(256,2,down3) # (8x8) -> (4x4)
    down5 = downsampling(256,2,down4) # (4x4) -> (2x2)
    down6 = downsampling(256,2,down5) # (2x2) -> (1x1)


    # Upsampling stack
    up5 = upsampling(filters=256,stride=2,prev_layer=down6,skip_layer=down5) # (1x1) -> (2x2)
    up4 = upsampling(256,2,up5,down4) # (2x2) -> (4x4)
    up3 = upsampling(256,2,up4,down3) # (4x4) -> (8x8)
    up2 = upsampling(128,2,up3,down2) # (8x8) -> (16x16)
    up1 = upsampling(64,2,up2,down1) # (16x16) -> (32x32)
    up0 = upsampling(32,2,up1,down0) # (32x32) -> (64x64)
    
    # Model output is (64x64) with 2 color channels with values between -1 and 1
    model_output = layers.Conv2DTranspose(
        2,strides=1,kernel_size=4,padding='same',activation='tanh',kernel_initializer=init,use_bias=False)(up0)

    model = keras.models.Model(model_input,model_output)
    return model

In [6]:
def generator_loss(disc_output_generated,generator_output,color_channels):

    # Part of the loss is based on the discriminator output on fake images
    cross_entropy = losses.BinaryCrossentropy(from_logits=True,label_smoothing=0.1)

    # Part of the loss (scaled) is based on the difference between the
    # generated images and the original colored images
    mse = losses.MeanSquaredError()
    mse_scaler = 25

    # Generator wants the discriminator to classify the generated images as 1 (real)
    # Adersarial loss is the defference between all 1s and the actual discriminator output
    adversarial_loss = cross_entropy(tf.ones_like(disc_output_generated),disc_output_generated)
    mse_loss = mse(generator_output,color_channels) 

    # Return all three losses for tensorboard
    return adversarial_loss + (mse_scaler * mse_loss), adversarial_loss, mse_loss

In [7]:
# Create the generator and generator optimizer
generator = make_generator()

# Beta1 to decrease importance of previous batches
generator_optimizer = optimizers.Adam(2e-4,beta_1=0.5)
generator.compile()
generator.summary()
#keras.utils.plot_model(generator,show_shapes=True,to_file='generator.png',dpi=48)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 64, 64, 32)   512         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 64, 32)  128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 64, 64, 32)   0           ['batch_normalization[0][0]']

In [8]:
def make_discriminator():

    init = initializers.RandomNormal()

    gray_input = layers.Input(shape=(64,64,1))
    color_input = layers.Input(shape=(64,64,2))

    # Gray and colored inputs are combined
    concat_input = layers.concatenate([gray_input,color_input])

    # Downsampling stack
    down0 = downsampling(filters=64,stride=2,prev_layer=concat_input) # (64x64) -> (32x32)
    down1 = downsampling(128,2,down0)        # (32x32) -> (16x16)
    down2 = downsampling(256,2,down1)        # (16x16) -> (8x8)

    # Model output is (8x8) with unbounded values (no activation)
    model_output = layers.Conv2D(1,2,strides=1,padding='same',kernel_initializer=init,use_bias=False)(down2)

    model = keras.Model(inputs=[gray_input,color_input],outputs=model_output)
    return model

In [9]:
def discriminator_loss(disc_output_generated,disc_output_real):

    cross_entropy = losses.BinaryCrossentropy(from_logits=True,label_smoothing=0.1)

    # Discriminator wants to classify real images as 1 and generated
    # images as 0. Loss is the difference between the desired outputs
    # and the actual outputs
    real_loss = cross_entropy(tf.ones_like(disc_output_real),disc_output_real)
    generated_loss = cross_entropy(tf.zeros_like(disc_output_generated),disc_output_generated)

    total_loss = real_loss + generated_loss
    return total_loss

In [10]:
# Make the discriminator and discriminator optimizer
discriminator = make_discriminator()
discriminator_optimizer = optimizers.Adam(2e-4)
discriminator.compile()
discriminator.summary()
#keras.utils.plot_model(discriminator,show_shapes=True,to_file='discriminator.png',dpi=48)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 64, 64, 1)]  0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 64, 64, 2)]  0           []                               
                                                                                                  
 concatenate_6 (Concatenate)    (None, 64, 64, 3)    0           ['input_2[0][0]',                
                                                                  'input_3[0][0]']                
                                                                                                  
 conv2d_7 (Conv2D)              (None, 32, 32, 64)   3072        ['concatenate_6[0][0]']    

In [11]:
@tf.function
def train_step(gray_channel,color_channels):

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

        # Generate a batch of fake images
        generator_output = generator(gray_channel,training=True)

        # Get discriminator output on real and generated images
        disc_output_real = discriminator([gray_channel,color_channels],training=True)
        disc_output_generated = discriminator([gray_channel,generator_output],training=True)

        # Calculate loss for generator and discriminator based on discriminator outputs
        gen_loss, adversarial_loss, mse_loss = generator_loss(disc_output_generated,generator_output,color_channels)
        disc_loss = discriminator_loss(disc_output_generated,disc_output_real)

    # Calculate generator gradients and train generator
    generator_gradients = gen_tape.gradient(gen_loss,generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(generator_gradients,generator.trainable_variables))

    # Calculate disciminator gradients and train discriminator
    discriminator_gradients = disc_tape.gradient(disc_loss,discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients,discriminator.trainable_variables))

    # Update tensorboard losses
    gen_loss_tracker(gen_loss)
    gen_adv_tracker(adversarial_loss)
    gen_mse_tracker(mse_loss)
    disc_loss_tracker(disc_loss)

In [12]:
def train_model(epochs,dataset):
    
    step_counter = 0

    # Progress bar/tracking
    for _ in tqdm(range(epochs)):

        for gray_channel,color_channels in dataset:
            train_step(gray_channel,color_channels)

            # Tensorboard logging for training
            with summary_writer.as_default():
                tf.summary.scalar('Generator_loss',gen_loss_tracker.result(),step=step_counter)
                tf.summary.scalar('Generator_adversarial_loss',gen_adv_tracker.result(),step=step_counter)
                tf.summary.scalar('Generator_mse_loss',gen_mse_tracker.result(),step=step_counter)
                tf.summary.scalar('Discriminator_loss',disc_loss_tracker.result(),step=step_counter)
            
            gen_loss_tracker.reset_states()
            gen_adv_tracker.reset_states()
            gen_mse_tracker.reset_states()
            disc_loss_tracker.reset_states()
            step_counter += 1

In [13]:
#train_model(epochs=200,dataset=train_dataset)
#generator.save('models/dusty/generator.h5')
#discriminator.save('models/dusty/discriminator.h5')
dusty_gen = keras.models.load_model('models/1-9-23/dusty/generator.h5')

In [14]:
# Empty lists for the frames for 3 gifs
train_folder = '../shows/episode3/'
save_folder = 'resources/video/'

<h1>Train Results</h1>

In [15]:
grayscale_frames = []
colorized_frames = []
original_frames = []

# 256 Frames is roughly 30 seconds of video
train_batch = utils.image_loader(directory=train_folder,
    batch_size=256,training=False,sort_files=True).__getitem__(2)

train_output = dusty_gen(train_batch[0],training=False)

for frame_number, gray_frame in enumerate(train_batch[0]):

    grayscale_dummy_color = np.dstack([np.zeros_like(gray_frame),
        np.zeros_like(gray_frame)])

    grayscale_frames.append(utils.lab_to_rgb_combine_channels(
        l_channel=gray_frame,
        ab_channels=grayscale_dummy_color
    ))

    colorized_frames.append(utils.lab_to_rgb_combine_channels(
        l_channel=gray_frame,
        ab_channels=train_output[frame_number]
    ))

    original_frames.append(utils.lab_to_rgb_combine_channels(
        l_channel=gray_frame,
        ab_channels = train_batch[1][frame_number]
    ))

with imageio.get_writer(save_folder + 'train_grayscale.mp4',mode='I',fps=8,quality=10) as writer:
    for frame in grayscale_frames:
        writer.append_data(frame)

with imageio.get_writer(save_folder + 'train_colorized.mp4',mode='I',fps=8,quality=10) as writer:
    for frame in colorized_frames:
        writer.append_data(frame)

with imageio.get_writer(save_folder + 'train_original.mp4',mode='I',fps=8,quality=10) as writer:
    for frame in original_frames:
        writer.append_data(frame)


<h3>Grayscale</h3>

In [16]:
Video('resources/video/train_grayscale.mp4',width=256)

<h3>Colorized using Model</h3>

In [17]:
Video('resources/video/train_colorized.mp4',width=256)

<h3>Original Color</h3>

In [18]:
Video('resources/video/train_original.mp4',width=256)